In [238]:
#Importing relevant libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [239]:
#Change the working directory
import os
os.chdir('C:\\Users\\HP\\Desktop\\')

In [240]:
#Create a dataframe
df = pd.read_csv('House Price.csv')

In [242]:
IV

'SalePrice'

In [ ]:
#Key data analysis
len(df)
df.head()
df.isnull().any()
df.isnull().sum()

In [252]:
#check for missing value, unique etc
FileNameDesc = pd.DataFrame(columns = ['column_name','missing_count','percent_missing','unique_count'])
for col in list(df.columns.values):
    sum_missing = df[col].isnull().sum()
    percent_missing = sum_missing/len(df)*100
    uniq_count = (df.groupby([col])[col].count()).count()
    FileNameDesc = FileNameDesc.append({'column_name':col,'missing_count':sum_missing,
                                        'percent_missing':percent_missing,'unique_count':uniq_count},
                                       ignore_index = True)

In [ ]:
FileNameDesc

In [246]:
File_Col_Type = (df.dtypes).reset_index()

In [253]:
#merging Column Type with other information about dataset
FileNameDesc = FileNameDesc.merge(File_Col_Type, left_on = 'column_name', right_on = 'index', how = 'inner')

In [255]:
del FileNameDesc['index']
FileNameDesc = FileNameDesc.rename(index=str, columns = {0:"column_type"})

In [ ]:
FileNameDesc

In [251]:
#Drop column with more than 25% null values
df.drop(['Alley'], axis = 1, inplace = True)

In [262]:
#Apply Mode strategy to populate the categorical data
df.groupby('GarageType').agg({'GarageType': np.size})
df["GarageType"]=df["GarageType"].fillna('Attchd')
df["GarageType"] = df["GarageType"].astype('category')
df["GarageType"] = df["GarageType"].cat.codes

In [263]:
#Mean strategy to populate the quantitative variable
df["LotFrontage"].fillna(df["LotFrontage"].mean(), inplace=True)

In [264]:
#Median strategy to populate the quantitative variable
df["MasVnrArea"].fillna(df["MasVnrArea"].median(), inplace=True)

In [274]:
df = pd.get_dummies(df, columns=['GarageType'], drop_first=True)

In [276]:
df1=df['SalePrice']

In [279]:
df.drop(['SalePrice'], axis = 1, inplace = True)

In [283]:
df=pd.concat([df,df1],axis=1)

In [ ]:
#Key data analysis
len(df)
df.head()
df.isnull().any()
df.isnull().sum()

In [ ]:
#Analyse the selected variables 
sns.pairplot(df)

In [ ]:
df.corr()
sns.heatmap(df.corr(),annot=False)

In [292]:
x = df.iloc[:,1:13]
y = df.iloc[:,13:]

In [295]:
from sklearn.cross_validation import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 100) 

In [296]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm = lm.fit(x_train,y_train)   #lm.fit(input,output)

In [297]:
lm.coef_

array([[-1.89195406e+02,  2.92488563e-01,  5.77934108e+01,
         9.31370291e+01,  6.58204127e+01, -8.60850226e+00,
         1.17301334e+02,  1.09608841e+05,  8.00334258e+04,
         1.17818148e+05,  4.44584210e+04,  7.70242422e+04]])

In [298]:
coefficients = pd.concat([pd.DataFrame(x_train.columns),pd.DataFrame(np.transpose(lm.coef_))], axis = 1)

In [300]:
lm.intercept_

array([-89484.77986412])

In [301]:
y_pred = lm.predict(x_test)

In [303]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.7252823600649496

In [304]:
import statsmodels.api as sma
x_train = sma.add_constant(x_train) ## let's add an intercept (beta_0) to our model
x_test = sma.add_constant(x_test) 

In [ ]:
import statsmodels.formula.api as sm
lm2 = sm.OLS(y_train,x_train).fit()
lm2.summary()

In [306]:
y_pred2 = lm2.predict(x_test) 

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
[variance_inflation_factor(x_train.values, j) for j in range(x_train.shape[1])]

In [ ]:
def calculate_vif(x):
    thresh = 5.0
    output = pd.DataFrame()
    k = x.shape[1]
    vif = [variance_inflation_factor(x.values, j) for j in range(x.shape[1])]
    for i in range(1,k):
        print("Iteration no.")
        print(i)
        print(vif)
        a = np.argmax(vif)
        print("Max VIF is for variable no.:")
        print(a)
        if vif[a] <= thresh :
            break
        if i == 1 :          
            output = x.drop(x.columns[a], axis = 1)
            vif = [variance_inflation_factor(output.values, j) for j in range(output.shape[1])]
        elif i > 1 :
            output = output.drop(output.columns[a],axis = 1)
            vif = [variance_inflation_factor(output.values, j) for j in range(output.shape[1])]
    return(output)
train_out = calculate_vif(x_train) 

In [ ]:
train_out.head()

In [ ]:
x_test.head()
x_test.drop(["LotFrontage","1stFlrSF","GrLivArea",'GarageArea'],axis = 1,inplace = True)
x_test.head()

In [ ]:
import statsmodels.api as sma
import statsmodels.formula.api as sm
train_out = sma.add_constant(train_out) ## let's add an intercept (beta_0) to our model
X_test = sma.add_constant(x_test)
lm2 = sm.OLS(y_train,train_out).fit()
lm2.summary()